In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [2]:
import pickle
from functools import reduce
import re
from tqdm import tqdm

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.utils import pad_sequences

from transformers import AutoTokenizer, AutoConfig, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

from datasets import load_dataset
import evaluate

In [3]:
path = '/content/drive/MyDrive/tr_project/'

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
all_sentences = pickle.load(open(f'{path}all_sentences_processed.pkl', 'rb'))
all_labels = pickle.load(open(f'{path}all_labels_processed.pkl', 'rb'))

In [6]:
len(all_sentences)

327325

In [7]:
def apos_pos_analysis(texts):
  i_s = []
  max_i = 0
  for text in texts:
    words = nltk.word_tokenize(text)
    for word in words:
      try:
        i = word.index("'")
        i = len(word) - i
        if i > max_i:
          max_i = i
          print(f'{word}: {i}')
        i_s.append(i)
      except:
        pass

  print(max(i_s))

In [8]:
# i've not known. 91 31 110
# i've not known. 91 31 110, 3, 0, 0
# ive not known. x 31 110, 3, 0, 0

In [9]:
def find_word_from_i(sent, i):
  j = i
  k = i
  while j > 0 and sent[j] != ' ':
    j -= 1

  while k < len(sent) and sent[k] != ' ':
    k += 1

  return sent[j:k].strip()

In [10]:
find_word_from_i(all_sentences[15], 1)

"i've"

In [11]:
def index_safe(s, c):
  try:
    return s.index(c)
  except:
    return -1

def count(s, c):
  x = 0
  for i in range(len(s)):
    if s[i] == c:
      x += 1
  return  x

In [12]:
def label_seq(sent, labels):
  new_seq = []
  label_i = 0
  char_c = 0
  reg_tok = nltk.RegexpTokenizer('[\w\'"]+|[.,;:?!\-\(\)]')
  seq = reg_tok.tokenize(sent)
  label_seq = [0] * len(seq)

  for i, c in enumerate(seq):
    try:
      word_len = len(c)
      j = index_safe(c, "'")
      if j != -1:
        j_ = len(c) - j
        if j_ <= 6:
          label_seq[i] = j_

        label_i += 1
      
      j = count(c, '"')
      if j > 0:
        label_i += 1

      if j == 1:
        j = index_safe(c, '"')
        if j == 0:
          label_seq[i] = 7
        else:
          label_seq[i] = 8
      elif j > 1:
        label_seq[i] = 9
      new_seq.append(c.replace('"', '').replace("'", ""))
      
      if label_i == len(labels):
        new_seq[i+1:] = seq[i+1:]
        break

    except:
      new_seq.append(c)

    char_c += word_len + 1

  
  return new_seq, label_seq


index = 180_000
res = label_seq(all_sentences[index], all_labels[index])

all_sentences[index], res[0], str(res[1]), len(res[1])

('The video for the third single from Bangerz, "Adore You", was posted on Vevo on December 26, 2013.',
 ['The',
  'video',
  'for',
  'the',
  'third',
  'single',
  'from',
  'Bangerz',
  ',',
  'Adore',
  'You',
  ',',
  'was',
  'posted',
  'on',
  'Vevo',
  'on',
  'December',
  '26',
  ',',
  '2013',
  '.'],
 '[0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]',
 22)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
tokenizer(res[0], is_split_into_words=True).word_ids(), len(tokenizer(res[0], is_split_into_words=True).word_ids())

([None,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  7,
  7,
  8,
  9,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  None],
 28)

In [15]:
seq_labels = [label_seq(s, l) for s, l in zip(all_sentences, all_labels)]
sentences = [s for s, l in seq_labels]
labels = [l for s, l in seq_labels]

In [16]:
padded_labels = pad_sequences(labels, maxlen=500)
np.unique(padded_labels, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32),
 array([163144450,     19381,    235105,     34977,      2717,      4329,
             3637,    100661,     67909,     49334]))

In [17]:
train_seq_labels, test_seq_labels = train_test_split(seq_labels, test_size=0.1, stratify=[max(l) for l in labels])

In [18]:
df = pd.DataFrame(train_seq_labels, columns=['text', 'label'])
df.to_json("train_data.json", orient="records", lines=True)

In [19]:
df = pd.DataFrame(test_seq_labels, columns=['text', 'label'])
df.to_json("test_data.json", orient="records", lines=True)

In [20]:
dataset = load_dataset("json", data_files={"train": "train_data.json", "test": "test_data.json"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-dbea49cc4190b019/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 294592
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 32733
    })
})

In [22]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

In [23]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            if label > 0:
              label = 2*label - 1
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            if label > 0:
              label *= 2
            
            new_labels.append(label)

    return new_labels

In [24]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["text"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["label"]
    new_labels = []

    for i, labels in enumerate(all_labels):
      word_ids = tokenized_inputs.word_ids(i)
      new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [25]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True, load_from_cache_file=False)
tokenized_dataset = tokenized_dataset.remove_columns(['label', 'text'])

  0%|          | 0/295 [00:00<?, ?ba/s]

  0%|          | 0/33 [00:00<?, ?ba/s]

In [26]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 294592
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 32733
    })
})

In [27]:
dataset['train']['text'][90], dataset['train']['label'][90], tokenized_dataset['train']['input_ids'][90], tokenized_dataset['train']['labels'][90]

(['Seymour',
  'Skinner',
  ',',
  'principal',
  'of',
  'the',
  'school',
  'Chief',
  'Wiggum',
  ',',
  'incompetent',
  'chief',
  'of',
  'police',
  'whos',
  'son',
  'is',
  'Ralph',
  '.'],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
 [101,
  13572,
  18835,
  117,
  3981,
  1104,
  1103,
  1278,
  2534,
  160,
  6512,
  19524,
  117,
  1107,
  8178,
  12924,
  3452,
  2705,
  1104,
  2021,
  1150,
  1116,
  1488,
  1110,
  6197,
  119,
  102],
 [-100,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  0,
  0,
  0,
  0,
  -100])

In [28]:
tokenizer(dataset['train']['text'][90], is_split_into_words=True).word_ids(), tokenizer.tokenize(dataset['train']['text'][90], is_split_into_words=True)

([None,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  8,
  8,
  9,
  10,
  10,
  10,
  10,
  11,
  12,
  13,
  14,
  14,
  15,
  16,
  17,
  18,
  None],
 ['Seymour',
  'Skinner',
  ',',
  'principal',
  'of',
  'the',
  'school',
  'Chief',
  'W',
  '##ig',
  '##gum',
  ',',
  'in',
  '##com',
  '##pet',
  '##ent',
  'chief',
  'of',
  'police',
  'who',
  '##s',
  'son',
  'is',
  'Ralph',
  '.'])

In [29]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [30]:
seqeval = evaluate.load("seqeval")

In [31]:
base_classes = np.arange(0, 10)
classes = np.concatenate([base_classes * 2, base_classes[1:] * 2 - 1])
classes = np.sort(classes)
classes = list(map(int, classes))
classes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [32]:
id2label = {
    0: 'O'
}
for i in range(1, len(classes), 2):
  if i < 13:
    pre = 'AP'
  else:
    pre = 'QU'
  id2label[classes[i]] = f'B-{pre}-{base_classes[i//2] + 1}'
  id2label[classes[i+1]] = f'I-{pre}-{base_classes[i//2] + 1}'


label2id = { v:k for k,v in id2label.items() }

In [33]:
id2label

{0: 'O',
 1: 'B-AP-1',
 2: 'I-AP-1',
 3: 'B-AP-2',
 4: 'I-AP-2',
 5: 'B-AP-3',
 6: 'I-AP-3',
 7: 'B-AP-4',
 8: 'I-AP-4',
 9: 'B-AP-5',
 10: 'I-AP-5',
 11: 'B-AP-6',
 12: 'I-AP-6',
 13: 'B-QU-7',
 14: 'I-QU-7',
 15: 'B-QU-8',
 16: 'I-QU-8',
 17: 'B-QU-9',
 18: 'I-QU-9'}

In [34]:
label2id

{'O': 0,
 'B-AP-1': 1,
 'I-AP-1': 2,
 'B-AP-2': 3,
 'I-AP-2': 4,
 'B-AP-3': 5,
 'I-AP-3': 6,
 'B-AP-4': 7,
 'I-AP-4': 8,
 'B-AP-5': 9,
 'I-AP-5': 10,
 'B-AP-6': 11,
 'I-AP-6': 12,
 'B-QU-7': 13,
 'I-QU-7': 14,
 'B-QU-8': 15,
 'I-QU-8': 16,
 'B-QU-9': 17,
 'I-QU-9': 18}

In [35]:
model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(id2label), id2label=id2label, label2id=label2id)
model

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [36]:
label_list = list(label2id.keys())

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    res = {}

    for c in base_classes[1:]:
      pre = 'AP' if c < 7 else 'QU'
      label_type = f"{pre}-{c}"
      res[f"{label_type}_f1"] = results[label_type]['f1']
      res[f"{label_type}_number"] = results[label_type]['number']
      res["overall_f1"] = results["overall_f1"]

    return res

In [42]:
train_path = '/content/drive/MyDrive/tr_project/bert_train/'

In [43]:
training_args = TrainingArguments(
    output_dir=train_path,
    learning_rate=3e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    gradient_accumulation_steps=2, 
    gradient_checkpointing=True,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [44]:
!nvidia-smi

Wed Jan 11 06:52:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |   1018MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
ckpt_path = '/content/drive/MyDrive/tr_project/bert_train/checkpoint-'

In [45]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 294592
  Num Epochs = 10
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 2
  Total optimization steps = 11510
  Number of trainable parameters = 107734291
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Ap-1 F1,Ap-1 Number,Overall F1,Ap-2 F1,Ap-2 Number,Ap-3 F1,Ap-3 Number,Ap-4 F1,Ap-4 Number,Ap-5 F1,Ap-5 Number,Ap-6 F1,Ap-6 Number,Qu-7 F1,Qu-7 Number,Qu-8 F1,Qu-8 Number,Qu-9 F1,Qu-9 Number
1,0.080000,0.069178,0.637613,1901,0.841335,0.969620,23456,0.957234,3492,0.529532,273,0.702576,424,0.574405,355,0.680446,8416,0.639332,6918,0.752282,4966
2,0.061000,0.064823,0.682359,1901,0.854223,0.973789,23456,0.969923,3492,0.636917,273,0.763218,424,0.680739,355,0.705203,8416,0.665597,6918,0.777904,4966
3,0.047100,0.064785,0.685061,1901,0.858490,0.973559,23456,0.971004,3492,0.668033,273,0.807468,424,0.737143,355,0.709782,8416,0.684673,6918,0.776595,4966
4,0.037700,0.069551,0.700271,1901,0.859980,0.975290,23456,0.973888,3492,0.698413,273,0.822144,424,0.729475,355,0.710668,8416,0.685616,6918,0.779323,4966
5,0.030500,0.071262,0.705538,1901,0.861866,0.976543,23456,0.972179,3492,0.693712,273,0.808559,424,0.724796,355,0.717813,8416,0.691018,6918,0.783228,4966
6,0.025200,0.076192,0.708436,1901,0.861651,0.976916,23456,0.976050,3492,0.700186,273,0.826446,424,0.744056,355,0.716655,8416,0.692057,6918,0.779249,4966
7,0.021300,0.083368,0.709338,1901,0.859915,0.977706,23456,0.974975,3492,0.709804,273,0.831962,424,0.755798,355,0.714311,8416,0.694140,6918,0.771431,4966
8,0.018400,0.086098,0.711697,1901,0.863505,0.977503,23456,0.976092,3492,0.715105,273,0.827751,424,0.759669,355,0.720608,8416,0.695556,6918,0.780169,4966
9,0.016600,0.089608,0.719125,1901,0.862933,0.977505,23456,0.975778,3492,0.717054,273,0.831354,424,0.739946,355,0.719532,8416,0.696290,6918,0.777834,4966
10,0.015100,0.092150,0.720000,1901,0.862627,0.977841,23456,0.975784,3492,0.715105,273,0.825623,424,0.744939,355,0.717861,8416,0.695696,6918,0.780674,4966


***** Running Evaluation *****
  Num examples = 32733
  Batch size = 128
Saving model checkpoint to /content/drive/MyDrive/tr_project/bert_train/checkpoint-1151
Configuration saved in /content/drive/MyDrive/tr_project/bert_train/checkpoint-1151/config.json
Model weights saved in /content/drive/MyDrive/tr_project/bert_train/checkpoint-1151/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/tr_project/bert_train/checkpoint-1151/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/tr_project/bert_train/checkpoint-1151/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 32733
  Batch size = 128
Saving model checkpoint to /content/drive/MyDrive/tr_project/bert_train/checkpoint-2302
Configuration saved in /content/drive/MyDrive/tr_project/bert_train/checkpoint-2302/config.json
Model weights saved in /content/drive/MyDrive/tr_project/bert_train/checkpoint-2302/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/

TrainOutput(global_step=11510, training_loss=0.0392461610775114, metrics={'train_runtime': 20831.234, 'train_samples_per_second': 141.418, 'train_steps_per_second': 0.553, 'total_flos': 1.5917179002291034e+17, 'train_loss': 0.0392461610775114, 'epoch': 10.0})